# Recurrent Neural Network - Step by Step

Recurrent Neural Networks (RNN) are very effective for Natural Language Processing and other sequence tasks because they have "memory." They can read inputs $x^{\langle t \rangle}$ (such as words) one at a time, and remember some contextual information through the hidden layer activations that get passed from one time step to the next. This allows a unidirectional (one-way) RNN to take information from the past to process later inputs. A bidirectional (two-way) RNN can take context from both the past and the future, much like Marty McFly. 

**Notation**:
- Superscript $[l]$ denotes an object associated with the $l^{th}$ layer. 
- Superscript $(i)$ denotes an object associated with the $i^{th}$ example. 
- Superscript $\langle t \rangle$ denotes an object at the $t^{th}$ time step.   
- Subscript $i$ denotes the $i^{th}$ entry of a vector.

<a name='0'></a>
## Packages

In [ ]:
import numpy as np
from rnn_utils import *

<a name='1'></a>
## 1 - Forward Propagation

In this example, $T_x = T_y$. 

<center><img src="images/RNN.png" width="70%" height="70%"></center>
<caption><center><b>Figure 1</b>: Basic RNN model</center></caption>

### Dimensions of input $x$

#### Input with $n_x$ number of units
* For a single time step of a single input example, $x^{(i) \langle t \rangle }$ is a one-dimensional input vector  
* The notation $n_x$ is used here to denote the number of units in a single time step of a single training example

#### Time steps of size $T_{x}$
* A recurrent neural network has multiple time steps, which you'll index with $t$.

#### Batches of size $m$
* You'll use $m$ to denote the number of training examples  
* The shape of a mini-batch is $(n_x,m,T_x)$

#### 3D Tensor of shape $(n_{x},m,T_{x})$
* The 3-dimensional tensor $x$ of shape $(n_x,m,T_x)$ represents the input $x$ that is fed into the RNN

### Dimensions of hidden state $a$

* The hidden state for a single training example is a vector of length $n_{a}$
* If you include a mini-batch of $m$ training examples, the shape of a mini-batch is $(n_{a},m)$
* When you include the time step dimension, the shape of the hidden state is $(n_{a}, m, T_x)$

### Dimensions of prediction $\hat{y}$
* $\hat{y}$ is a 3D tensor of shape $(n_{y}, m, T_{y})$
    * $n_{y}$: number of units in the vector representing the prediction
    * $m$: number of examples in a mini-batch
    * $T_{y}$: number of time steps in the prediction

<a name='1-1'></a>
### 1.1 - RNN Cell Foward

We can think of the recurrent neural network as the repeated use of a single cell.

<center><img src="images/rnn_step_forward_figure2_v3a.png" width="70%" height="70%"></center>

<caption><p><b>Figure 2</b>: Basic RNN cell. Takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (previous hidden state containing information from the past), and outputs $a^{\langle t \rangle}$ which is given to the next RNN cell and also used to predict $\hat{y}^{\langle t \rangle}$</p></caption>

In [ ]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m)
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
            Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
            Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
            Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
            ba -- Bias, numpy array of shape (n_a, 1)
            by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    # computing next activation state
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba, dtype='float32')
    
    # computing output of the current cell
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    
    # storing values needed for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

<a name='1-2'></a>
### 1.2 - Forward Pass 

- A recurrent neural network (RNN) is a repetition of the RNN cell.
- Each cell takes two inputs at each time step:
    - $a^{\langle t-1 \rangle}$: The hidden state from the previous cell
    - $x^{\langle t \rangle}$: The current time step's input data <br><br>
    
- It has two outputs at each time step:
    - A hidden state ($a^{\langle t \rangle}$)
    - A prediction ($y^{\langle t \rangle}$) <br><br>
    
- The weights and biases $(W_{aa}, b_{a}, W_{ax}, b_{x})$ are re-used each time step 
    - They are maintained between calls to `rnn_cell_forward` in the 'parameters' dictionary


<center><img src="images/rnn_forward_sequence_figure3_v3a.png"></center>
<caption><center><b>Figure 3</b>: Basic RNN. The input sequence $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  is carried over $T_x$ time steps. The network outputs $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>


In [ ]:
def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network.

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    caches = []

    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape

    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))

    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        
        a[:,:,t] = a_next
        y_pred[:,:,t] = yt_pred
        
        caches.append(cache)
    
    # storing values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

<a name='2'></a>    
## 2 - Backpropagation

In modern deep learning frameworks, you only have to implement the forward pass, and the framework takes care of the backward pass, so most deep learning engineers do not need to bother with the details of the backward pass.

Note that this notebook does not implement the backward path from the Loss 'J' backwards to 'a'. This would have included the dense layer and softmax, which are a part of the forward path. This is assumed to be calculated elsewhere and the result passed to `rnn_backward` in 'da'. It is further assumed that loss has been adjusted for batch size (m) and division by the number of examples is not required here.

<a name='2-1'></a>    
### 2.1 - RNN Backward Cell

The following figure shows the operations of an RNN backward cell:

<center><img src="images/rnn_backward_overview_3a_1.png" width="50%" height="50%"></center>

<caption><p><b>Figure 6</b>: The RNN cell's backward pass. Just like in a fully-connected neural network, the derivative of the cost function $J$ backpropagates through the time steps of the RNN by following the chain rule from calculus. Internal to the cell, the chain rule is also used to calculate $(\frac{\partial J}{\partial W_{ax}},\frac{\partial J}{\partial W_{aa}},\frac{\partial J}{\partial b})$ to update the parameters $(W_{ax}, W_{aa}, b_a)$. The operation can utilize the cached results from the forward path.</p></caption>

<center><img src="images/rnn_cell_backward_3a_c.png" width="50%" height="50%"></center>
<caption><p><b>Figure 7</b>: This implementation of `rnn_cell_backward` does **not** include the output dense layer and softmax which are included in `rnn_cell_forward`. $da_{next}$ is $\frac{\partial{J}}{\partial a^{\langle t \rangle}}$ and includes loss from previous stages and current stage output logic. The addition shown in green will be part of your implementation of `rnn_backward`.  </p></caption>

#### Equations:


$$ a^{\langle t \rangle} = \tanh(W_{ax} x^{\langle t \rangle} + W_{aa} a^{\langle t-1 \rangle} + b_{a})$$
<br>
$$\frac{\partial \tanh(x)}{\partial x} = 1 - \tanh^2(x)$$
<br>
$${dtanh} = da_{next} * ( 1 - \tanh^2(W_{ax}x^{\langle t \rangle} + W_{aa} a^{\langle t-1 \rangle} + b_{a}))$$
<br>
$${dW_{ax}} = dtanh \cdot x^{\langle t \rangle T}\tag{1}$$
<br>
$$dW_{aa} = dtanh \cdot a^{\langle t-1 \rangle T}\tag{2}$$
<br>
$$db_a = \sum_{batch}dtanh\tag{3}$$
<br>
$$dx^{\langle t \rangle} = { W_{ax}}^T \cdot dtanh\tag{4}$$
<br>
$$da_{prev} = { W_{aa}}^T \cdot dtanh\tag{5}$$

In [ ]:
def rnn_cell_backward(da_next, cache):
    """
    Implements the backward pass for the RNN-cell (single time-step).

    Arguments:
    da_next -- Gradient of loss with respect to next hidden state
    cache -- python dictionary containing useful values (output of rnn_cell_forward())

    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradients of input data, of shape (n_x, m)
                        da_prev -- Gradients of previous hidden state, of shape (n_a, m)
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dba -- Gradients of bias vector, of shape (n_a, 1)
    """
    

    (a_next, a_prev, xt, parameters) = cache
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    dz = da_next*(1 - np.square(np.tanh(a_next), dtype='float32'))

    dxt = np.dot(Wax.transpose(), dz)
    dWax = np.dot(dz, xt.transpose())

    da_prev = np.dot(Waa.transpose(), dz)
    dWaa = np.dot(dz, a_prev.transpose())

    dba = np.sum(dz, axis=1, keepdims=True)

    gradients = {"dxt": dxt, "da_prev": da_prev, "dWax": dWax, "dWaa": dWaa, "dba": dba}
    
    return gradients

<a name='2-2'></a>    
### 2.2 - Backward Pass

In [ ]:
def rnn_backward(da, caches):
    """
    Implement the backward pass for a RNN over an entire sequence of input data.

    Arguments:
    da -- Upstream gradients of all hidden states, of shape (n_a, m, T_x)
    caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient w.r.t. the input data, numpy-array of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t the initial hidden state, numpy-array of shape (n_a, m)
                        dWax -- Gradient w.r.t the input's weight matrix, numpy-array of shape (n_a, n_x)
                        dWaa -- Gradient w.r.t the hidden state's weight matrix, numpy-arrayof shape (n_a, n_a)
                        dba -- Gradient w.r.t the bias, of shape (n_a, 1)
    """

    (caches, x) = caches
    (a1, a0, x1, parameters) = caches[0]
    
    n_a, m, T_x = da.shape
    n_x, m = x1.shape 
    
    # initializing the gradients 
    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
    
    # Loop through all the time steps
    for t in reversed(range(T_x)):
        gradients = rnn_cell_backward(da_prevt + np.reshape(da[:,:,t], (n_a, m)), caches[t])

        dxt = gradients["dxt"]
        da_prevt = gradients["da_prev"]
        dWaxt = gradients["dWax"]
        dWaat = gradients["dWaa"]
        dbat = gradients["dba"]
        
        dx[:, :, t] = dxt  
        dWax += dWaxt 
        dWaa += dWaat  
        dba += dbat  
        
    da0 = da_prevt

    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba}
    
    return gradients